<a href="https://colab.research.google.com/github/shiissaa/MAT421/blob/main/ModuleA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

9.1 Base-N and Binary

Write a function my_bin_2_dec(b) where b is binary number represented by a list of ones and zeros. The last element of b represents the coefficient of 2^0, the second-to-last element of b represents the coefficient of 2^1, and so on. The output variable, d, should be the decimal representation of b. The test cases are provided below.

In [163]:
def my_bin_2_dec(b):
  d = 0
  # Exponent value
  n = 0
  # Reverse to start at least significant bit
  b.reverse()
  for i in b:
    if (i == 1):
      d += 2**n
    n+=1
  return d

# Output: 33554431
my_bin_2_dec([1]*25)

33554431

Write a function my_dec_2_bin(d), where d is a positive integer in decimal, and b is the binary representation of d. The output b must be a list of ones and zeros, and the leading term must be a 1 unless the decimal input value is 0. The test cases are provided below.



In [162]:
# Used to format output to match the output in the textbook
import ast

def my_dec_2_bin(d):
  b = []
  # Check for case when input is 0
  if (d == 0):
    b.append("0")
  else:
    while (d > 0):
      if (d % 2 == 1):
        b.append("1")
      else:
        b.append("0")
      # Divide by 2 in each loop until !d>0, checking for remainder during each loop
      d = d//2
  # Reverse for proper order
  b.reverse()
  b = [ast.literal_eval(i) for i in b]
  return b

# Output: [1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1]
my_dec_2_bin(2097)

[1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1]

Use the two functions you wrote in problems 1 and 2 to compute d = my_bin_2_dec(my_dec_2_bin(12654)). Do you get the same number?



In [161]:
d = my_bin_2_dec(my_dec_2_bin(12654))
print (d)

12654


Answer: Yes, the number is the same.


Write a function my_bin_adder(b1,b2), where b1, b2 and the output variable b are binary numbers represented as in problem 1. The output variable should be computed as b = b1 + b2. Do not use your functions from problems 1 and 2 to write this function (i.e., do not convert b1 and b2 to decimal; add them, and then convert the result back to binary). This function should be able to accept inputs b1 and b2 of any length (i.e., very long binary numbers), and b1 and b2 may not necessarily be the same length.



In [164]:
import ast

def my_bin_adder(b1, b2):
  carry = 0
  # Ensure all binary values have the same number of bits
  length = max(len(b1), len(b2))
  b = [None]*length
  # Initial reverse to pad zeros on the correct side
  b1.reverse()
  b2.reverse()
  # Pad with zeros using new length
  b1 = (b1 + [0] * length)[:length]
  b2 = (b2 + [0] * length)[:length]
  # Reverse back after zeros have been added
  b1.reverse()
  b2.reverse()
  # Iterate through the sum's list, starting at the least significant bit
  for i in range(length - 1, -1, -1):
    loopCarry = carry
    loopCarry += b1[i] + b2[i]
    # Check for remainder and find sum
    if (loopCarry % 2 == 1):
      b[i] = "1"
    else:
      b[i] = "0"
    # Determine if carry needs to be stored
    if (loopCarry < 2):
      carry = 0
    else:
      carry = 1
  # Determine if there was "overflow"
  if (carry != 0):
    b.reverse()
    b.append("1")
    b.reverse()

  b = [ast.literal_eval(i) for i in b]
  return b

# Output: [1, 0, 1, 1]
my_bin_adder([1, 1, 0], [1, 0, 1])

[1, 0, 1, 1]

In [159]:
# Binary multiplication
# 29    x    9 =   261
# 11101 x 1001 = 11101 +
#               000000 +
#              0000000 +
#             11101000 = 100000101

261 == 2**8 + 2**2 + 2**0

True

9.2 Floating Point Numbers

Converting between base10 and IEEE754

```
What is 17.0(base10) in IEEE754?
17.0 is positive, so sign bit = 0
The largest power of two smaller than 17.0 is 16, so the exponent value is 4
The characteristic is 4 + 1023 = 1027(base10), 10000000011(base2)
The fraction is 17/16 - 1 = 0.0625 or 0001...
Thus, 17.0(base10) = 0 10000000011 0001000000000000000000000000000000000000000000000000(IEEE754)

What is  0 10000000010 1110000000000000000000000000000000000000000000000000(IEEE754) in base10?

The leftmost bit is a 0, so the value is positive
The exponent value is 2**10 + 2**1 = 1026 - 1023 = 3
The fraction is 2**(-1) + 2**(-2) + 2**(3) = 0.875
Thus, the base10 representation of the value is 1 * 2**3 * (1 + .875) = 15.0

```



Adding or subtracting a 1 to the binary fraction portion of the IEEE754 number will also give the next largest or smallest number, respectively.
The distance between two numbers is called the gap. The gap grows as the number being represented grows.
A gap can be computed using a function.
If a value is added or subtracted to/from a number, and that value is less than half the gap from one number to the immediate next, then the same number will be returned.

In [174]:
import numpy as np

print(np.spacing(1e9))
1e9 == (1e9 + np.spacing(1e9/3))

1.1920928955078125e-07


True

Subnormal numbers occur during special cases, namely when the exponent has a value of 0. If the exponent has a value of 0, then the leading 1 in the fraction becomes a 0. When the exponent has a value of 2047 and the fraction is nonzero, the result is an undefined number. When e = 2047 and both the sign and fraction portions are 0, the result is positive infinity. When e = 2047 and the fraction portion is 0, but the sign portion is 1, the result is negative infinity. The largest and smallest IEEE754 64-bit values can be verified by Python.

In [180]:
import sys

max = (2**1023)*((1 + sum(0.5**np.arange(1, 53))))
max == sys.float_info.max

True

In [181]:
min = (2**(-1022))*(1+0)
min == sys.float_info.min

True

Overflow occurs when a calculation results in a number larger than what can be represented. The result is assigned to inf. When there is not sufficient precision for storage, operations are equivalent to adding 0.
Underflow occurs when a calculation results in a number smaller than the smallest subnormal number. The result is assigned to 0.

In [182]:
# Overflow
sys.float_info.max == sys.float_info.max + 2

True

In [183]:
sys.float_info.max + sys.float_info.max

inf

In [185]:
# Underflow
0 == 2**(-1075)

True

In [184]:
print(2**(-1075))
print(2**(-1074))

0.0
5e-324


 9.3 Round-off Errors

Round-off errors refer to the difference between an approximation used in computation and its true value. One common round-off error is a representation error, occurring when using the approximation (finite number of digits) of an infinite number e.g. pi, 1/3. On separate occassions, it might not be a big deal, but these errors can accumulate. This can be seen using Python.

In [189]:
print (1.9 - 1.832)
1.9 - 1.832 == 0.068

0.06799999999999984


False

Accumulation can be seen in instances where the same operation is performed multiple times.

In [209]:
# Case where accumulation does not occur due to not enough operations
2 + 1/3 - 1/3

2.0

In [211]:
# Function to loop operations
def loop(iterations):
  result = 2
  for i in range(iterations):
    result += 1/3
  for i in range(iterations):
    result -= 1/3
  return result

In [217]:
# 1000 loops
loop(1000)

2.000000000000006

In [216]:
# 10000 loops
loop(10000)

2.0000000000001164

In [215]:
# 100000 loops
loop(100000)

2.000000000001921